In [1]:
import os, sys, glob, re, time
from os.path import exists

import geopandas as gpd 
import pandas as pd
import numpy as np 
import fiona
from shapely.geometry import Point, LineString, Polygon, shape, MultiPoint
import shapely.wkt
from shapely.ops import cascaded_union

from xrspatial import zonal_stats 
import rasterio
from rasterio.plot import show
from rasterio import features, mask
from rasterio.features import shapes
from osgeo import gdal, osr, ogr, gdal_array

In [10]:
ProjectFolder = os.getcwd()
print(ProjectFolder)

C:\Users\wb527163\GEO_Cdrive_Grace\Sahel_Cdrive\exploratory_climate_study\Agriculture


### MapSPAM

Before running the script: Downloaded the data for each year and added desired production data to respective year folders.

In [7]:
# Make a list of rainfed, low-input agricultural production rasters for each year.
spam2000 = [i for i in os.listdir(r'spam2000_prod_rain_lowinput/') if i.endswith('_L.tif')]
spam2005 = [i for i in os.listdir(r'spam2005_prod_rain_lowinput/') if i.endswith('_L.tif')]
spam2010 = [i for i in os.listdir(r'spam2010_prod_rain_lowinput/') if i.endswith('_L.tif')]
spam2017 = [i for i in os.listdir(r'spam2017_prod_rain_lowinput/') if i.endswith('_L.tif')]

print(spam2000, spam2005, spam2010, spam2017)

['spam2000v3r7_production_BANP_L.tif', 'spam2000v3r7_production_BARL_L.tif', 'spam2000v3r7_production_BEAN_L.tif', 'spam2000v3r7_production_CASS_L.tif', 'spam2000v3r7_production_COFF_L.tif', 'spam2000v3r7_production_COTT_L.tif', 'spam2000v3r7_production_GROU_L.tif', 'spam2000v3r7_production_MAIZ_L.tif', 'spam2000v3r7_production_OFIB_L.tif', 'spam2000v3r7_production_OOIL_L.tif', 'spam2000v3r7_production_OPUL_L.tif', 'spam2000v3r7_production_OTHE_L.tif', 'spam2000v3r7_production_POTA_L.tif', 'spam2000v3r7_production_RICE_L.tif', 'spam2000v3r7_production_SORG_L.tif', 'spam2000v3r7_production_SOYB_L.tif', 'spam2000v3r7_production_SUGB_L.tif', 'spam2000v3r7_production_SUGC_L.tif', 'spam2000v3r7_production_SWPY_L.tif', 'spam2000v3r7_production_WHEA_L.tif'] ['SPAM2005V3r2_global_P_TL_ACOF_L.tif', 'SPAM2005V3r2_global_P_TL_BANA_L.tif', 'SPAM2005V3r2_global_P_TL_BARL_L.tif', 'SPAM2005V3r2_global_P_TL_BEAN_L.tif', 'SPAM2005V3r2_global_P_TL_CASS_L.tif', 'SPAM2005V3r2_global_P_TL_CHIC_L.tif', 'SPA

In [ ]:
CheckContents = [rasterio.open(r"spam2000_prod_rain_lowinput/spam2000v3r7_production_BANP_L.tif"), 
               rasterio.open(r"spam2005_prod_rain_lowinput/SPAM2005V3r2_global_P_TL_ACOF_L.tif"), 
               rasterio.open(r"spam2010_prod_rain_lowinput/spam2010V2r0_global_P_ACOF_L.tif"),
               rasterio.open(r"spam2017_prod_rain_lowinput/spam2017V2r1_SSA_P_ACOF_L.tif")]

for item in CheckContents:
    print(item.name, "\nBands= ", item.count, "\nWxH= ", item.width, "x", item.height, "\n\n")

stats = []
for item in CheckContents:
    band = item.read(1)
    stats.append({
        'raster': item.name,
        'min': band.min(),
        'mean': band.mean(),
        'median': np.median(band),
        'max': band.max()})

# Show stats for each channel
print("\n", stats)

In [20]:
CheckContents = None
band = None
item = None

In [17]:
def SimplifyFileNames(folder):
    for count, filename in enumerate(os.listdir(folder)):
            dst = ''.join(['Crop', str(count), '.tif'])
            src =f"{folder}/{filename}"  # foldername/filename, if .py file is outside folder
            dst =f"{folder}/{dst}"

            os.rename(src, dst)

In [21]:
SimplifyFileNames(folder = 'spam2000_prod_rain_lowinput')
SimplifyFileNames(folder = 'spam2005_prod_rain_lowinput')
SimplifyFileNames(folder = 'spam2010_prod_rain_lowinput')
SimplifyFileNames(folder = 'spam2017_prod_rain_lowinput')

In [ ]:
# Open command line for this:

_P_		production

*_L	rainfed low inputs portion of crop or crop group


cd C:\Users\wb527163\GEO_Cdrive_Grace\Sahel_Cdrive\exploratory_climate_study\Agriculture\spam2000_prod_rain_lowinput

# Create an empty raster using the specs of one of the rasters in the folder.
gdal_create -if Crop1.tif -burn 0 ProdSum.tif

# Add all rasters in the folder with ProdSum.tif as the output location.
for %f in (*.tif) do gdal_calc.py -A %f -B result.tif --outfile=ProdSum.tif --calc=A+B

gdal_calc.py -A *.tif --outfile=ProdSum.tif --calc="numpy.sum(A,axis=0)"